### Ijaz Ahmad

In [2]:
import random
import numpy as np
import tensorflow as tf

2025-01-07 13:14:07.109051: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 13:14:07.358606: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 13:14:07.597443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736237647.814236    6133 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736237647.872767    6133 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [4]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [5]:
text = text[300000:800000]

In [6]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [7]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [8]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [11]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool_)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

In [12]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [13]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

2025-01-07 13:16:54.451785: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/ahmad/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(lr=0.01))

# model.fit(x, y, batch_size=256, epochs=4)
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01)
)

model.fit(x, y, batch_size=256, epochs=4)


Epoch 1/4


2025-01-07 13:18:19.912245: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 259980240 exceeds 10% of free system memory.


651/651 ━━━━━━━━━━━━━━━━━━━━ 188s 285ms/step - loss: 2.4524
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 170s 261ms/step - loss: 1.7608
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 177s 272ms/step - loss: 1.5994
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 201s 270ms/step - loss: 1.5162


In [16]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [18]:
# print(generate_text(300, 0.2))
# print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
# print(generate_text(300, 0.6))
# print(generate_text(300, 0.7))
# print(generate_text(300, 0.8))

ch substance of a grief hath twenty shade.

duke of aumerle:
my haster come, that my here her bear me.

baltla:
of his proce my grace that madam and my sours
and true of this he shope their least not this me
with the house of warwick as this be thine,
the his her much and the said the comfort it present.
what shall the mishand he well of 
